In [2]:
import pandas as pd
import numpy as np
import os
import glob

In [3]:
path1 = "/data/projects/PSB/Output"

In [4]:
# Find files matching the pattern
pattern = "_candidates.tsv"
matching_files = glob.glob(os.path.join(path1, "**", f"*{pattern}"), recursive=True)

In [5]:
matching_files

['/data/projects/PSB/Output/Donor/Donor_candidates.tsv',
 '/data/projects/PSB/Output/Acceptor/Acceptor_candidates.tsv',
 '/data/projects/PSB/Output/nonTATA/nonTATA_candidates.tsv']

In [5]:
tata = pd.read_csv(matching_files[0], sep="\t")
don  = pd.read_csv(matching_files[1], sep="\t")
ntata = pd.read_csv(matching_files[2], sep="\t")
acc = pd.read_csv(matching_files[3], sep="\t")

In [6]:
print(tata.shape, ntata.shape, acc.shape, don.shape)

(5695866, 17) (594945, 17) (1936700, 18) (1004448, 19)


In [7]:
acc.columns

Index(['index', 'chr', 'strand', 'Acceptor_coordinates',
       'Transcript_coordinates', 'dbsnp_id', 'varinat_start', 'variant_end',
       'ref_neucleotide', 'alternative_neucleotide', 'reference_seq',
       'alt_seq', 'Ensemble_Transcript_ID', 'Ref_probab', 'Alt_probab',
       'ScoreChange', 'LogOddRatio', 'variant_length'],
      dtype='object')

In [8]:
data_frames = {
    'tata': tata,
    'ntata': ntata,
    'acc': acc,
    'don': don
}

# subset df
subset_data_frames = {}
selected_columns = ['chr', 'strand', 'Ensemble_Transcript_ID',
       'Transcript_coordinates', 'dbsnp_id', 'varinat_start', 'variant_end', 'Ref_probab', 'Alt_probab', 'ScoreChange', 'LogOddRatio']

# Subset df
subset_data_frames = {}
for df_name, df in data_frames.items():
    # Determine the coordinate column to use ('Donor_coordinates' or 'Acceptor_coordinates')
    coordinate_column = 'Donor_coordinates' if 'Donor_coordinates' in df.columns else 'Acceptor_coordinates' if 'Acceptor_coordinates' in df.columns else None
    # Add the coordinate_column to the selected columns if it's present
    columns_to_use = selected_columns + [coordinate_column] if coordinate_column else selected_columns
    # Subset the DataFrame using the columns that are actually present in the DataFrame
    subset_data_frames[df_name] = df[df.columns.intersection(columns_to_use)]

In [14]:
subset_data_frames["acc"].head(2)

,chr,strand,Acceptor_coordinates,Transcript_coordinates,dbsnp_id,varinat_start,variant_end,Ensemble_Transcript_ID,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
0,chr1,+,24445144-24445223,24415803-24472976,rs1450817635,24445172,24445174,ENST00000003912,0.999804,0.001602,-0.998007,21.603005
1,chr1,+,24445144-24445223,24415814-24466310,rs1450817635,24445172,24445174,ENST00000339255,0.999804,0.001602,-0.998007,21.603005


In [13]:
# unique number of variants before merging with tsptr
subset_data_frames['acc'].drop_duplicates(subset=['chr', 'strand', 'varinat_start', 'variant_end']).shape

(213634, 12)

In [16]:
# read tsptr
tsptr = pd.read_csv('/home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/tsptr_all.csv').drop("Unnamed: 0", axis =1)
tsptr = tsptr[['transcript', 'tissue', 'expression']]
tsptr.head(2)

,transcript,tissue,expression
0,ENST00000000233,all,HouseTrans
1,ENST00000000412,all,HouseTrans


In [17]:
for df_name, df in subset_data_frames.items():
    df = df.copy()
    
    # Replace occurrences of "NA," "NullTrans," or NaN with "others"
    df.replace(["NA", "NullTrans"], "Others", inplace=True)
    df.fillna("others", inplace=True)
    
    # add tsptr information
    new_df = df.merge(tsptr, left_on='Ensemble_Transcript_ID', right_on='transcript', how='left')
    
    subset_data_frames[df_name] = new_df

In [19]:
subset_data_frames['acc'].shape

(1936700, 15)

In [20]:
# replace donor or acceptor_coordinates with func_coordinates for coherent naming
for df_name, df in subset_data_frames.items():
    if "Donor_coordinates" in df.columns:
        df.rename(columns={"Donor_coordinates": "Func_coordinates"}, inplace=True)
    if "Acceptor_coordinates" in df.columns:
        df.rename(columns={"Acceptor_coordinates": "Func_coordinates"}, inplace=True)
    subset_data_frames[df_name] = df


In [21]:
subset_data_frames['acc'].head(3)

,chr,strand,Func_coordinates,Transcript_coordinates,dbsnp_id,varinat_start,variant_end,Ensemble_Transcript_ID,Ref_probab,Alt_probab,ScoreChange,LogOddRatio,transcript,tissue,expression
0,chr1,+,24445144-24445223,24415803-24472976,rs1450817635,24445172,24445174,ENST00000003912,0.999804,0.001602,-0.998007,21.603005,NaN,NaN,NaN
1,chr1,+,24445144-24445223,24415814-24466310,rs1450817635,24445172,24445174,ENST00000339255,0.999804,0.001602,-0.998007,21.603005,ENST00000339255,"Fallopian_Tube, Lung, Pancreas, Prostate, Sali...",TenhTrans
2,chr1,+,24445144-24445223,24415813-24454824,rs1450817635,24445172,24445174,ENST00000358028,0.999804,0.001602,-0.998007,21.603005,NaN,NaN,NaN


In [22]:
# take unique for total variants on chr strand varinat_start variant_end
unique_columns = ['chr', 'strand', 'varinat_start', 'variant_end']

unique_variants = {}
for df_name, df in subset_data_frames.items():
    unique_df = df.drop_duplicates(subset=unique_columns)
    unique_variants[df_name] = unique_df

In [23]:
# Iterate through each DataFrame in the dictionary
for df_name, df in unique_variants.items():
    print(f"The shape of DataFrame '{df_name}' is {df.shape}")

The shape of DataFrame 'tata' is (358026, 15)
The shape of DataFrame 'ntata' is (42464, 15)
The shape of DataFrame 'acc' is (213634, 15)
The shape of DataFrame 'don' is (122234, 15)


In [24]:
# unique variants
for df_name, df in unique_variants.items():
    unique_variants[df_name] = df.replace(["WideTrans", "NullTrans"], "Others").fillna("Others")

# total counts
total_counts = pd.Series(dtype='int')
for df_name, df in unique_variants.items():
    expression_counts = df['expression'].value_counts()
    expression_counts['Total'] = expression_counts.sum()
    output = expression_counts.to_frame(name='Counts').reset_index()
    output.columns = ['Expression', 'Counts']
    output_str = output.to_string(index=False, justify='left')
    print(f"'{df_name}'\n{output_str}\n")

'tata'
Expression  Counts
    Others 265879 
HouseTrans  36418 
  TspTrans  28998 
 TenhTrans  26731 
     Total 358026 

'ntata'
Expression  Counts
    Others 33626  
  TspTrans  3429  
HouseTrans  2840  
 TenhTrans  2569  
     Total 42464  

'acc'
Expression  Counts
    Others 153531 
  TspTrans  20579 
 TenhTrans  20113 
HouseTrans  19411 
     Total 213634 

'don'
Expression  Counts
    Others  87087 
 TenhTrans  12293 
HouseTrans  11862 
  TspTrans  10992 
     Total 122234 



In [132]:
path1 = "/home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/"

# save
for df_name, df in unique_variants.items():
    filename = path1 + f"{df_name}.csv"
    df.to_csv(filename, index=False)
    print(f"Saved DataFrame '{df_name}' to {filename}")

Saved DataFrame 'tata' to /home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/tata.csv
Saved DataFrame 'ntata' to /home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/ntata.csv
Saved DataFrame 'acc' to /home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/acc.csv
Saved DataFrame 'don' to /home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/don.csv


In [25]:
# take unique of func site
# unique func site
unique_columns = ['chr', 'strand', 'Func_coordinates']

unique_variants_func = {}
for df_name, df in unique_variants.items():
    unique_df = df.drop_duplicates(subset=unique_columns)
    unique_variants_func[df_name] = unique_df

for df_name, df in unique_variants_func.items():
    unique_variants_func[df_name] = df.replace(["WideTrans", "NullTrans"], "Others").fillna("Others")

# total counts
total_counts = pd.Series(dtype='int')
for df_name, df in unique_variants_func.items():
    expression_counts = df['expression'].value_counts()
    expression_counts['Total'] = expression_counts.sum()
    output = expression_counts.to_frame(name='Counts').reset_index()
    output.columns = ['Expression', 'Counts']
    output_str = output.to_string(index=False, justify='left')
    print(f"'{df_name}'\n{output_str}\n")

'tata'
Expression  Counts
    Others 72876  
HouseTrans  8574  
  TspTrans  8212  
 TenhTrans  7257  
     Total 96919  

'ntata'
Expression  Counts
    Others 22475  
  TspTrans  2344  
HouseTrans  2040  
 TenhTrans  1773  
     Total 28632  

'acc'
Expression  Counts
    Others 44109  
  TspTrans  5853  
 TenhTrans  5693  
HouseTrans  5357  
     Total 61012  

'don'
Expression  Counts
    Others 51761  
 TenhTrans  7142  
HouseTrans  6912  
  TspTrans  6502  
     Total 72317  



In [5]:
# merge clinvar ahead
path1 = "/home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/dbsnp/"

# Find files matching the pattern
pattern = ".csv"
matching_files = glob.glob(os.path.join(path1, "**", f"*{pattern}"), recursive=True)
matching_files

['/home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/acc.csv',
 '/home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/ntata.csv',
 '/home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/tata.csv',
 '/home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/don.csv']

In [8]:
acc = pd.read_csv(matching_files[0])
ntata  = pd.read_csv(matching_files[1])
tata = pd.read_csv(matching_files[2])
don = pd.read_csv(matching_files[3])

In [9]:
print(tata.shape, ntata.shape, acc.shape, don.shape)

(358026, 15) (42464, 15) (213634, 15) (122234, 15)


In [14]:
clinvar = pd.read_csv('/home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/clinvar_snv_sorted.bed', header=None, sep="\t")
clinvar.columns = ["chr", "variant_start", "variant_end", "dbsnp_id"]
clinvar.head()

,chr,variant_start,variant_end,dbsnp_id
0,chr1,925951,925952,rs1640863258
1,chr1,930187,930188,rs770001898
2,chr1,930199,930200,rs368250686
3,chr1,930202,930203,rs767794127
4,chr1,930247,930248,rs41285790


In [15]:
data_frames_clin = {
    'tata': tata,
    'ntata': ntata,
    'acc': acc,
    'don': don
}

In [17]:
data_frames_clin['tata'].head(3)

,chr,strand,Func_coordinates,Ensemble_Transcript_ID,Transcript_coordinates,dbsnp_id,varinat_start,variant_end,Ref_probab,Alt_probab,ScoreChange,LogOddRatio,transcript,tissue,expression
0,chr1,-,171652644-171652733,ENST00000037502,171635417-171652688,rs931124801,171652687,171652688,0.999701,0.013452,-0.985954,17.903774,Others,Others,Others
1,chr1,-,53945655-53945744,ENST00000194214,53921561-53945699,rs1462348969,53945670,53945671,0.758328,0.004605,-0.571570,9.405620,ENST00000194214,all,HouseTrans
2,chr1,-,53945655-53945744,ENST00000194214,53921561-53945699,rs925379522,53945671,53945672,0.758328,0.091363,-0.505779,4.963791,ENST00000194214,all,HouseTrans


In [18]:
merged_clin = {
    key: df.merge(clinvar, left_on=['varinat_start', 'variant_end', 'chr'], right_on=['variant_start', 'variant_end', 'chr'], how='inner')
    for key, df in data_frames_clin.items()
}

In [19]:
# Iterate through each DataFrame in the dictionary
for df_name, df in merged_clin.items():
    print(f"The shape of DataFrame '{df_name}' is {df.shape}")

The shape of DataFrame 'tata' is (4080, 17)
The shape of DataFrame 'ntata' is (576, 17)
The shape of DataFrame 'acc' is (4927, 17)
The shape of DataFrame 'don' is (3238, 17)


In [22]:
# # unique variants
# for df_name, df in merged_clin.items():
#     unique_variants[df_name] = df.replace(["WideTrans", "NullTrans"], "Others").fillna("Others")

# total counts
total_counts = pd.Series(dtype='int')
for df_name, df in merged_clin.items():
    expression_counts = df['expression'].value_counts()
    expression_counts['Total'] = expression_counts.sum()
    output = expression_counts.to_frame(name='Counts').reset_index()
    output.columns = ['Expression', 'Counts']
    output_str = output.to_string(index=False, justify='left')
    print(f"'{df_name}'\n{output_str}\n")

'tata'
Expression  Counts
    Others 3021   
HouseTrans  512   
 TenhTrans  306   
  TspTrans  241   
     Total 4080   

'ntata'
Expression  Counts
    Others 450    
HouseTrans  47    
  TspTrans  45    
 TenhTrans  34    
     Total 576    

'acc'
Expression  Counts
    Others 3509   
 TenhTrans  566   
HouseTrans  497   
  TspTrans  355   
     Total 4927   

'don'
Expression  Counts
    Others 2300   
 TenhTrans  326   
HouseTrans  324   
  TspTrans  288   
     Total 3238   



In [23]:
path1 = "/home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/clinvar/"

# save
for df_name, df in merged_clin.items():
    filename = path1 + f"{df_name}.csv"
    df.to_csv(filename, index=False)
    print(f"Saved DataFrame '{df_name}' to {filename}")

Saved DataFrame 'tata' to /home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/clinvar/tata.csv
Saved DataFrame 'ntata' to /home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/clinvar/ntata.csv
Saved DataFrame 'acc' to /home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/clinvar/acc.csv
Saved DataFrame 'don' to /home/campus.stonybrook.edu/psurana/projects/PSB_2024/v3/Output_data/clinvar/don.csv


In [24]:
merged_clin['don'].head(2)

,chr,strand,Func_coordinates,Transcript_coordinates,dbsnp_id_x,varinat_start,variant_end,Ensemble_Transcript_ID,Ref_probab,Alt_probab,ScoreChange,LogOddRatio,transcript,tissue,expression,variant_start,dbsnp_id_y
0,chr1,+,11960729-11960808,11934717-11975537,rs775007891,11960732,11960733,ENST00000196061,0.853494,0.318220,-0.456853,3.641704,ENST00000196061,all,HouseTrans,11960732,rs775007891
1,chr1,-,91698049-91698128,91695490-91712363,rs2228363,91698088,91698089,ENST00000470600,0.923592,0.456204,-0.431676,3.848853,Others,Others,Others,91698088,rs2228363


In [27]:
merged_clin['don'].shape

(3238, 17)

In [31]:
subset_df = merged_clin['don'][merged_clin['don']['dbsnp_id_x'] != merged_clin['don']['dbsnp_id_y']]
subset_df.head()

,chr,strand,Func_coordinates,Transcript_coordinates,dbsnp_id_x,varinat_start,variant_end,Ensemble_Transcript_ID,Ref_probab,Alt_probab,ScoreChange,LogOddRatio,transcript,tissue,expression,variant_start,dbsnp_id_y
9,chr1,-,29200476-29200555,29200324-29203170,rs1239954306,29200514,29200515,ENST00000464511,0.868477,0.355287,-0.445693,3.582839,Others,Others,Others,29200514,rs778879465
15,chr1,+,99870831-99870910,99850489-99924020,rs1571243862,99870831,99870832,ENST00000294724,0.733034,0.137456,-0.436579,4.106853,ENST00000294724,Muscle,TspTrans,99870831,rs1294297593
27,chr1,+,205059644-205059723,205058347-205060520,rs1360269746,205059649,205059650,ENST00000639015,0.894348,0.036751,-0.766990,7.793553,Others,Others,Others,205059649,rs1318994065
40,chr1,-,220182678-220182757,220181551-220195547,rs750379226,220182722,220182723,ENST00000693454,0.936607,0.247937,-0.645012,5.485919,Others,Others,Others,220182722,rs1553275687
44,chr1,+,156115236-156115315,156114251-156139293,rs794728603,156115246,156115247,ENST00000361308,0.860855,0.351332,-0.438625,3.513825,Others,Others,Others,156115246,rs556237236


In [33]:
# take unique of func site
# unique func site
unique_columns = ['chr', 'strand', 'Func_coordinates']

unique_variants_func = {}
for df_name, df in merged_clin.items():
    unique_df = df.drop_duplicates(subset=unique_columns)
    unique_variants_func[df_name] = unique_df

# for df_name, df in unique_variants_func.items():
#     unique_variants_func[df_name] = df.replace(["WideTrans", "NullTrans"], "Others").fillna("Others")

# total counts
total_counts = pd.Series(dtype='int')
for df_name, df in unique_variants_func.items():
    expression_counts = df['expression'].value_counts()
    expression_counts['Total'] = expression_counts.sum()
    output = expression_counts.to_frame(name='Counts').reset_index()
    output.columns = ['Expression', 'Counts']
    output_str = output.to_string(index=False, justify='left')
    print(f"'{df_name}'\n{output_str}\n")

'tata'
Expression  Counts
    Others 1760   
HouseTrans  281   
 TenhTrans  171   
  TspTrans  163   
     Total 2375   

'ntata'
Expression  Counts
    Others 372    
HouseTrans  39    
  TspTrans  38    
 TenhTrans  31    
     Total 480    

'acc'
Expression  Counts
    Others 2128   
 TenhTrans  357   
HouseTrans  283   
  TspTrans  218   
     Total 2986   

'don'
Expression  Counts
    Others 1741   
 TenhTrans  257   
HouseTrans  234   
  TspTrans  180   
     Total 2412   

